In [1]:
import scipy as sp
from scipy import stats
import numpy as np 
import pandas as pd
from FileExporter import create_download_link
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from msresist.mining_bioreplicates import MergeDfbyMean, AvsBacrossCond, AvsBvsCacrossCond
from msresist.plsr import FilteringOutPeptides, MeasuredVsPredicted_LOOCVplot
from msresist.comp_estimator import MyOwnKMEANS
from sklearn.cross_decomposition import PLSRegression
from sklearn.pipeline import Pipeline
from msresist.sequence_analysis import trim, Distance, GenerateFastaFile
from Bio import SeqIO, AlignIO
import re

In [2]:
# Varaibles: X phoshopeptides
X = pd.read_csv('./msresist/data/20180817_JG_AM_TMT10plex_R1_psms.csv', header=0)
header = pd.read_csv('./msresist/data/20180817_JG_AM_TMT10plex_R1_psms.csv', header=None).iloc[0,:]
treatments = pd.read_csv('./msresist/data/20180817_JG_AM_TMT10plex_R1_psms.csv', header=None).iloc[0,2:]

PN = []
x = list(map(lambda  v : PN.append(v), X.iloc[:,1]))

X_protnames, X_seqs = [], []
x = list(map(lambda  v : X_protnames.append(v.split("OS")[0]), X.iloc[:,1]))
y = list(map(lambda  v : X_seqs.append(v), X.iloc[:,0]))

X_data = X.iloc[:,2:]

# Filtering out insignificant phosphopeptides 
Xf = FilteringOutPeptides(X, header)
Xf_data = np.transpose(Xf.iloc[:, 2:])
Xf_seqs = Xf.iloc[:,0]
Xf_protnames = Xf.iloc[:,1]

# Observations: Y
Y = pd.read_csv('./msresist/data/ydata.csv', header=0)
Y_cv = Y.iloc[:10, 2]

X.shape

(287, 12)

In [3]:
MS_seqs = open( "./msresist/data/MS_seqs.txt", "r")
proteome = open('./msresist/data/proteome_uniprot.txt', "r")

In [4]:
# count = 0
# for rec1 in SeqIO.parse(MS_seqs, "fasta"):
#     MS_seq = rec1.seq
#     MS_seqU = rec1.seq.split("-")[0].upper()
#     MS_name = rec1.id
#     count += 1
    
# print(count)

In [5]:
DictProtToSeq_UP = {}
for rec2 in SeqIO.parse(proteome, "fasta"):
    UP_seq = str(rec2.seq)
    UP_name = rec2.description.split("HUMAN ")[1].split(" OS")[0]
    DictProtToSeq_UP[UP_name] = str(UP_seq)


ExtSeq = []
counter = 0
for rec1 in SeqIO.parse(MS_seqs, "fasta"):
    MS_seq = str(rec1.seq)
    MS_seqU = str(rec1.seq.split("-")[0].upper())
    MS_name = str(rec1.description.split(" OS")[0])
    try:
        UP_seq = DictProtToSeq_UP[MS_name]
        if MS_seqU in UP_seq and MS_name == list(DictProtToSeq_UP.keys())[list(DictProtToSeq_UP.values()).index(UP_seq)]:
            counter += 1
            regexPattern = re.compile(MS_seqU)
            MatchObs = regexPattern.finditer(UP_seq)
            indices = []
            for i in MatchObs:
                indices.append(i.start())
                indices.append(i.end())
            ExtSeq.append(UP_seq[indices[0]:indices[1]])
    except:
        print("find ", MS_name, "in proteome_uniprot.txt and change name")
        pass
print(counter)
print(ExtSeq)

find  Protein LAP2 in proteome_uniprot.txt and change name
find  Cell differentiation protein RCD1 homolog in proteome_uniprot.txt and change name
find  Probable palmitoyltransferase ZDHHC20 in proteome_uniprot.txt and change name
find  Pseudopodium-enriched atypical kinase 1 in proteome_uniprot.txt and change name
find  Tyrosine-protein kinase SgK223 in proteome_uniprot.txt and change name
find  Protein LAP2 in proteome_uniprot.txt and change name
find  FYN-binding protein in proteome_uniprot.txt and change name
find  Calmodulin in proteome_uniprot.txt and change name
find  Aldose reductase in proteome_uniprot.txt and change name
278
['AAVPSGASTGIYEALELR', 'AAVPSGASTGIYEALELRDNDK', 'AAVPSGASTGIYEALELRDNDKTR', 'ADHQPLTEASYVNLPTIALCNTDSPLR', 'ADYDTLSLR', 'AEDGSVIDYELIDQDAR', 'AEERPTFDYLQSVLDDFYTATEGQYQQQP', 'AEFAEYASVDR', 'AEGSDVANAVLDGADCIMLSGETAKGDYPLEAVR', 'AESGPDLRYEVTSGGGGTSR', 'AITNNQYEIV', 'ALALGASTVMMGSLLAATTEAPGEYFFSDGIR', 'ALDYYMLR', 'ALLYLCGGDD', 'ALPQNDDHYVMQEHR', 'ALYDNVPEC

In [6]:
    if line1.startswith(">"):
        MS_protname = line1.split("[")[1].split("_")[0]
    else:
        MS_seq = line1.split("-")[0].upper()
        for line2 in proteome:
            if line2.startswith(">"):
                UP_protname = line2.split("_")[0].split("|")[2]
            else:
                UP_seq = line2.str()
                print(UP_seq)
                raise SystemExit
                
            elif MS_protname == UP_protname and MS_seq in line2:
                print("check")
                
                
                regexPattern = re.compile(MS_pept)
                MatchObs = regexPattern.finditer(line2)
                indices = []
                for i in MatchObs:
                    indices.append(i.start())
                    indices.append(i.end())
                ExtSeq.append(line2[indices[0]:indices[1]])
            

#     elif "y" not in line1.split("-")[0][0:5] and "y" not in line1.split("-")[0][-5:] and "t" not in line1.split("-")[0][0:5] and "t" not in line1.split("-")[0][-5:] and "s" not in line1.split("-")[0][0:5] and "s" not in line1.split("-")[0][-5:]:

SyntaxError: invalid syntax (<ipython-input-6-fea4a3564a65>, line 13)

In [ ]:
raise SystemExit

In [ ]:
seqs = []
labels = []
for seq_record in SeqIO.parse(proteome, "fasta"):
    seqs.append(seq_record.seq)
    labels.append(seq_record.id)

seqMat = np.array(seqs)
label = np.array(labels)

print(seqMat.shape)
print(label.shape)

sequence = seqMat[:, 0:20874]

In [ ]:
D_PAM = Distance("protein-sequences.fa", "PAM250")
distMat = D_PAM.dist_mat()